<a href="https://colab.research.google.com/github/dheeraj7000/Colab-NB-GenAI/blob/main/Jokes_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00


In [11]:
from datasets import load_dataset

dataset = load_dataset("Fraser/short-jokes")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for Fraser/short-jokes contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Fraser/short-jokes
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  wa

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

In [21]:
dataset['train']['text'][0]

'[me narrating a documentary about narrators] ""I can\'t hear what they\'re saying cuz I\'m talking""\n'

In [22]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 231657
    })
})

In [27]:
text = ""
for i in dataset['train']['text']:
  text += i
text = text[:1115394]
print(len(text))

1115394


In [29]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz|~
94


In [30]:
#create a mapping from characters to integers
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s : [stoi[ch] for ch in s]
decode = lambda l : ''.join([itos[i] for i in l])

In [35]:
print(encode("hello! there"))
print(decode(encode("hello! there")))

[73, 70, 77, 77, 80, 2, 1, 85, 73, 70, 83, 70]
hello! there


In [37]:
#encode entire text dataset
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([60, 78, 70,  1, 79, 66, 83, 83, 66, 85, 74, 79, 72,  1, 66,  1, 69, 80,
        68, 86, 78, 70, 79, 85, 66, 83, 90,  1, 66, 67, 80, 86, 85,  1, 79, 66,
        83, 83, 66, 85, 80, 83, 84, 62,  1,  3,  3, 42,  1, 68, 66, 79,  8, 85,
         1, 73, 70, 66, 83,  1, 88, 73, 66, 85,  1, 85, 73, 70, 90,  8, 83, 70,
         1, 84, 66, 90, 74, 79, 72,  1, 68, 86, 91,  1, 42,  8, 78,  1, 85, 66,
        77, 76, 74, 79, 72,  3,  3,  0, 53, 70, 77, 77, 74, 79, 72,  1, 78, 90,
         1, 69, 66, 86, 72, 73, 85, 70, 83,  1, 72, 66, 83, 77, 74, 68,  1, 74,
        84,  1, 72, 80, 80, 69,  1, 71, 80, 83,  1, 90, 80, 86, 15,  1, 40, 80,
        80, 69,  1, 74, 78, 78, 86, 79, 70,  1, 84, 90, 84, 85, 70, 78,  1, 66,
        79, 69,  1, 76, 70, 70, 81, 84,  1, 81, 70, 84, 85, 84,  1, 66, 88, 66,
        90, 15,  1, 53, 74, 68, 76, 84, 13,  1, 78, 80, 84, 82, 86, 74, 85, 80,
        84, 13,  1, 87, 66, 78, 81, 74, 83, 70, 84, 15, 15, 15,  1, 78, 70, 79,
      

In [46]:
n = int(0.9 * len(data))
train_data = data[:n]
test_data = data[n:]

In [40]:
block_size = 8
train_data[:block_size+1]

tensor([60, 78, 70,  1, 79, 66, 83, 83, 66])

In [41]:
 x = train_data[:block_size]
 y = train_data[1:block_size+1]
 for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target : {target}")

when input is tensor([60]) the target : 78
when input is tensor([60, 78]) the target : 70
when input is tensor([60, 78, 70]) the target : 1
when input is tensor([60, 78, 70,  1]) the target : 79
when input is tensor([60, 78, 70,  1, 79]) the target : 66
when input is tensor([60, 78, 70,  1, 79, 66]) the target : 83
when input is tensor([60, 78, 70,  1, 79, 66, 83]) the target : 83
when input is tensor([60, 78, 70,  1, 79, 66, 83, 83]) the target : 66


In [44]:
x

tensor([60, 78, 70,  1, 79, 66, 83, 83])

In [45]:
y

tensor([78, 70,  1, 79, 66, 83, 83, 66])

In [57]:
torch.manual_seed(1337)
batch_size = 4
blocj_sie = 8

def get_batch(split):
  data = train_data if split == 'train' else val_dat
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_split(split='train')
print(xb)
print(yb)

tensor([[ 1, 85, 73, 70,  1, 85, 83, 66],
        [70, 85,  1, 85, 73, 70,  1, 71],
        [86, 84,  1, 68, 77, 80, 88, 79],
        [ 1, 47, 74, 85, 83, 80, 72, 70]])
tensor([[85, 73, 70,  1, 85, 83, 66, 74],
        [85,  1, 85, 73, 70,  1, 71, 86],
        [84,  1, 68, 77, 80, 88, 79,  1],
        [47, 74, 85, 83, 80, 72, 70, 79]])


In [58]:
text[:1115394]

'[me narrating a documentary about narrators] ""I can\'t hear what they\'re saying cuz I\'m talking""\nTelling my daughter garlic is good for you. Good immune system and keeps pests away. Ticks, mosquitos, vampires... men.\nI\'ve been going through a really rough period at work this week It\'s my own fault for swapping my tampax for sand paper.\nIf I could have dinner with anyone, dead or alive... ...I would choose alive. -B.J. Novak-\nTwo guys walk into a bar. The third guy ducks.\nWhy can\'t Barbie get pregnant? Because Ken comes in a different box. Heyooooooo\nWhy was the musician arrested? He got in treble.\nDid you hear about the guy who blew his entire lottery winnings on a limousine? He had nothing left to chauffeur it.\nWhat do you do if a bird shits on your car? Don\'t ask her out again.\nHe was a real gentlemen and always opened the fridge door for me\nTelling my daugthers date that ""she has lice and its very contagious the closer you get to her."" *Correct way to parent.\nW

In [59]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 94])
tensor(4.8725, grad_fn=<NllLossBackward0>)

l"";/+,F(&MGF|jw`dtJ) Qqrsh]?y6'0J<'qRW@xk3Ere~p\w<A_/Q3xCMkXF3+q_& WSDi-gxa^QO3gVzcO3I\VTNLs:d#L(hN


In [60]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [62]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

4.82966947555542


In [63]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


'~2;!\XYvJEns:@vB5` B7"GWo/1L _`b B9B$ dEN2sY6]A>^Q/PB%S/'>9~3H+La
.FW`9Y\^IL]K$a:].X@pYEX,91B6!S,\;oRoPq7<+%>x1]aF2B6(;(Bi*c+.kgUu,EuE`%+&
@la.;[Z9qQb?6Sf|i*-|&&ti-yVSLxsr03V-08d!W=3qR~-D],aVPZO,%`nPi$h9Ns&Dd$1P`d)I+l-^0! j%>(fgEgM-/VFwRW@I?jO/R-FTCh,Ri@H*;=b)P7A=!-fWas|Fe Z=4z5Njg+B|MWWQgfB$-SVEmaeIJow6s8/s5i5!u7wRrsx@UmJ
T!&2SyCKF@,I:Drsr~2l0'bz%Syx0Irsrw(cy:DX+%>BRkv"o:gh,K(B^~f*Q>8Gy]?FIGM/ltDrsh+L*<n_"Z9uB~+s%>9~L?iyXpi#LbAYe`zj^Q+uzz8ys
A/Eh@u6.V<t@lKq(acv
]?|oHqVb-fBK\wAic>A/\r Q.e&&WrNr


In [64]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [65]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [67]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# # wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# with open('input.txt', 'r', encoding='utf-8') as f:
#     text = f.read()

text = text

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

0.21347 M parameters
step 0: train loss 4.6835, val loss 4.6830
step 100: train loss 2.7617, val loss 2.7503
step 200: train loss 2.6318, val loss 2.6275
step 300: train loss 2.5354, val loss 2.5403
step 400: train loss 2.4768, val loss 2.4727
step 500: train loss 2.4126, val loss 2.4254
step 600: train loss 2.3689, val loss 2.3710
step 700: train loss 2.3220, val loss 2.3200
step 800: train loss 2.2775, val loss 2.2977
step 900: train loss 2.2479, val loss 2.2432
step 1000: train loss 2.1982, val loss 2.1885
step 1100: train loss 2.1772, val loss 2.1657
step 1200: train loss 2.1398, val loss 2.1599
step 1300: train loss 2.1236, val loss 2.1286
step 1400: train loss 2.0897, val loss 2.1063
step 1500: train loss 2.0785, val loss 2.0973
step 1600: train loss 2.0593, val loss 2.0667
step 1700: train loss 2.0526, val loss 2.0664
step 1800: train loss 2.0388, val loss 2.0364
step 1900: train loss 2.0194, val loss 2.0178
step 2000: train loss 1.9948, val loss 2.0206
step 2100: train loss 1.9

In [68]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


Gir find a cosseucticisus.
Did you hear peeat woman?
Feat on If you offuld you get righted rest, it ? Tirly and .... Diftachbly saying
""Sun,'t their faire my lost over tell the they me senting meanu? You will me to his Slivie in a live: Paokeu've anyove nest, ""the hable seed.
([Syric $5 Hamany: BYoom abs from with the weath he accalabactfs nic fonom? You heapoaminific sudaginis.
-Your goon mans 9 aked hrandma Becaused I'm fas hie. Uking U ghroigh to Curp and yourst people and weorld in seagu stirce? Youch Ropeet I got to startyunce want apolice of Bed almom in having fet my, my chists do meles womehing!
I've not twold coeot a girlacau danna abpoteolic atin tent me the rearm shef attubaftice? Haha, you add thell now meny seld are Throom's today my pean.
In ealine home beatiot in the wall, about for Reate in and when he was twogder aff and exicizity diked to me.
Atra
Jump a look on hot stace chind a sweats. Cloking so 4, is than a sin his fair dristeler.. the staude it should tatcher 